In [1]:
import numpy as np


dataset = np.random.randint(0, 11, size=(100, 100))
# Print the dataset
print(dataset)

[[ 8 10  6 ...  9  0  2]
 [ 0 10  3 ...  7  6  0]
 [ 0  9  5 ...  0  7  4]
 ...
 [ 2  6  5 ... 10  4  8]
 [ 2  6  8 ...  6  4  9]
 [ 5  2  1 ... 10 10  0]]


In [2]:
import pandas as pd
users = [f'user{i}' for i in range(1, 101)]
movies = [f'movie{i}' for i in range(1, 101)]
df = pd.DataFrame(dataset, index=users, columns=movies)

In [3]:
df.head()

,movie1,movie2,movie3,movie4,movie5,movie6,movie7,movie8,movie9,movie10,...,movie91,movie92,movie93,movie94,movie95,movie96,movie97,movie98,movie99,movie100
user1,8,10,6,1,9,7,1,10,10,8,...,6,10,8,10,8,8,4,9,0,2
user2,0,10,3,10,2,4,8,5,5,3,...,10,0,5,4,10,8,2,7,6,0
user3,0,9,5,6,10,9,2,8,1,6,...,9,1,9,3,9,8,6,0,7,4
user4,6,5,9,7,6,2,6,1,6,1,...,4,6,9,3,1,10,10,7,6,5
user5,0,10,8,5,1,0,7,5,6,4,...,10,7,4,5,10,10,5,0,2,6


In [4]:
# prompt: calculate the nearest neighbours for users for all movies

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Assuming 'df' is your DataFrame (as defined in the previous code)

# Reshape the data for NearestNeighbors
# We transpose because we want to find similar users based on their movie ratings
X = df.T.values

# Initialize NearestNeighbors model
knn = NearestNeighbors(metric='cosine', algorithm='brute')

# Fit the model
knn.fit(X)

# Example: Find 5 nearest neighbors for 'user1'
distances, indices = knn.kneighbors(X[0].reshape(1, -1), n_neighbors=6)  # Find 6, as the first neighbor is itself

# Print the results
print("Nearest neighbors for user1:")
for i in range(1, len(distances[0])):  # Exclude the user itself
  print(f"{df.columns[indices[0][i]]}: Distance = {distances[0][i]}")


# Function to find nearest neighbors for all users
def find_nearest_neighbors(user_index, k=6):  # Adjust k for desired neighbors
  distances, indices = knn.kneighbors(X[user_index].reshape(1, -1), n_neighbors=k)
  return distances, indices


# Example: Calculate for all users (excluding the user itself)
for i in range(len(users)):
  distances, indices = find_nearest_neighbors(i)
  print(f"Nearest neighbors for {users[i]}:")
  for j in range(1, len(distances[0])):
      print(f"{users[indices[0][j]]}: Distance = {distances[0][j]}")


Nearest neighbors for user1:
movie13: Distance = 0.20851906992691127
movie22: Distance = 0.21203592600021914
movie86: Distance = 0.21449574675114969
movie18: Distance = 0.21740851513155313
movie90: Distance = 0.21966426484806423
Nearest neighbors for user1:
user13: Distance = 0.20851906992691127
user22: Distance = 0.21203592600021914
user86: Distance = 0.21449574675114969
user18: Distance = 0.21740851513155313
user90: Distance = 0.21966426484806423
Nearest neighbors for user2:
user57: Distance = 0.21550996864674876
user65: Distance = 0.21907970272350608
user63: Distance = 0.22539122589765026
user97: Distance = 0.22601706655150267
user8: Distance = 0.22740333453903105
Nearest neighbors for user3:
user79: Distance = 0.22191382448262909
user4: Distance = 0.24842445853484607
user61: Distance = 0.26598905920924976
user85: Distance = 0.26835007033935754
user68: Distance = 0.2711008777825693
Nearest neighbors for user4:
user79: Distance = 0.20728318621982134
user22: Distance = 0.2131628134303

In [5]:
# prompt: generate user1 rating for movie 3 using cosine similarity

# Assuming 'df' and 'knn' are defined as in the previous code

def predict_rating(user, movie, k=5):
    """Predicts the rating of a movie for a given user using k-nearest neighbors."""

    user_index = df.index.get_loc(user)  # Get the index of the user
    movie_index = df.columns.get_loc(movie)  # Get the index of the movie

    distances, indices = knn.kneighbors(X[user_index].reshape(1, -1), n_neighbors=k + 1)  # Find k+1 nearest neighbors (excluding the user itself)


    # Calculate weighted average rating
    weighted_sum = 0
    similarity_sum = 0
    for i in range(1, len(distances[0])):  # Exclude user itself (index 0)
        neighbor_index = indices[0][i]
        similarity = 1 - distances[0][i]  # Cosine similarity
        weighted_sum += similarity * X[neighbor_index][movie_index]
        similarity_sum += similarity
    if similarity_sum > 0:
        predicted_rating = weighted_sum / similarity_sum
    else:
        predicted_rating = df[movie].mean()  # Fallback to mean if no similar users

    return predicted_rating

# Predict rating for user1 for movie3
user1_movie3_rating = predict_rating('user1', 'movie3')
print(f"Predicted rating for user1 on movie3: {user1_movie3_rating}")


Predicted rating for user1 on movie3: 3.5959822889026296


In [6]:
movie_df = pd.read_csv("/content/Movie Ratings.csv")

In [7]:
movie_df.head()

,UserId,MovieId,Rating,Timestamp
0,1,68646,10,1381620027
1,1,113277,10,1379466669
2,2,454876,8,1394818630
3,2,790636,7,1389963947
4,2,816711,8,1379963769


In [1]:
movie_df = pd.read_csv("/content/Movie Ratings.csv", index_col="UserId")
# Assuming 'rating' is the column for movie ratings.

# Group by 'userId'
grouped_df = movie_df.groupby(movie_df.index)

grouped_df.head()

NameError: name 'pd' is not defined

In [9]:


combined_data = {}


for user_id, group in grouped_df:
    combined_data[user_id] = {}
    for index, row in group.iterrows():
        movie_id = row['MovieId']
        rating = row['Rating']
        combined_data[user_id][movie_id] = rating


new_df = pd.DataFrame.from_dict(combined_data, orient='index')



In [10]:
new_df.describe()

,68646,113277,454876,790636,816711,1091191,1322269,1433811,1454468,1535109,...,89421,1692942,1904937,990413,211661,469318,1907614,1999192,3399154,2638984
count,206.000000,103.000000,766.000000,1073.000000,2144.000000,632.000000,277.000000,347.000000,2670.000000,1732.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,9.203883,8.699029,8.124021,8.069897,7.310634,7.957278,7.440433,7.763689,8.323970,8.330254,...,7.0,7.0,9.0,7.0,5.0,4.0,5.0,7.0,7.0,10.0
std,1.367634,1.251156,1.595426,1.090904,1.468143,1.438571,1.472289,1.224299,1.533291,1.175303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,3.000000,1.000000,1.000000,0.000000,1.000000,2.000000,2.000000,1.000000,1.000000,...,7.0,7.0,9.0,7.0,5.0,4.0,5.0,7.0,7.0,10.0
25%,9.000000,8.000000,8.000000,8.000000,7.000000,7.000000,7.000000,7.000000,8.000000,8.000000,...,7.0,7.0,9.0,7.0,5.0,4.0,5.0,7.0,7.0,10.0
50%,10.000000,9.000000,8.000000,8.000000,7.000000,8.000000,7.000000,8.000000,9.000000,8.000000,...,7.0,7.0,9.0,7.0,5.0,4.0,5.0,7.0,7.0,10.0
75%,10.000000,10.000000,9.000000,9.000000,8.000000,9.000000,8.000000,8.000000,9.000000,9.000000,...,7.0,7.0,9.0,7.0,5.0,4.0,5.0,7.0,7.0,10.0
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,7.0,7.0,9.0,7.0,5.0,4.0,5.0,7.0,7.0,10.0


In [ ]:
from sklearn.impute import KNNImputer
Y = new_df.T
imputer = KNNImputer(n_neighbors=5, metric='nan_euclidean')
Y_imputed = imputer.fit_transform(Y)
new_df_imputed = pd.DataFrame(Y_imputed, index=Y.index, columns=Y.columns)
new_df_imputed.head()
